### Part 1.

Electro-Poly is a leading maker of slip-rings. A $750,000 order has just been received and must be filled.  Constraints in filling the order are based on hours for wiring each unit and hours for harnessing each unit. 

|                            | Model 1 | Model 2 | Model 3 |
| :---                       | :----:  |    ---: |    ---: |
| # Ordered                  | 3000    | 2000    | 900     |
| Hours for Wiring / Unit    |2	       |1.5	     |3        |
| Hours of Harnessing / Unit |1	       |2	     |1        |
| Cost to Make               |50	   |83	     |130      |
| Cost to Buy                |61	   |97	     |145      |


The company has 10,000 hours of wiring capacity and 5,000 hours of harnessing capacity. It must either make or buy the various models to meet the demand.  Formulate this LP and solve using R, Python, GAMS, Go, or Excel.  Provide your complete code and share with your colleagues.

In [1]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, PULP_CBC_CMD

In [44]:
# Set up the variables
## Cannot have less than 0 of Model 1, 2, or 3
model_1_make = LpVariable("Model 1 to make", 0, 3000)  # upper limit is 3000
model_2_make = LpVariable("Model 2 to make", 0, 2000)
model_3_make = LpVariable("Model 3 to make", 0, 900)
model_1_buy = LpVariable("Model 1 to buy", 0, 3000)  
model_2_buy = LpVariable("Model 2 to buy", 0, 2000)
model_3_buy = LpVariable("Model 3 to buy", 0, 900)

# Set up the objective function type
prob = LpProblem("Slip Ring Cost Min Problem", LpMinimize)

# Set up the objective function
prob += (50*model_1_make + 83*model_2_make + 130*model_3_make + 
         61*model_1_buy + 97*model_2_buy + 145*model_3_buy, "Total cost")

# Set up the constraints
prob += (2*model_1_make + 1.5*model_2_make + 3*model_3_make <= 10000, "Wiring capacity hours")
prob += (1*model_1_make + 2*model_2_make + 1*model_3_make <= 5000, "Harnessing capacity hours")
prob += (1*model_1_make + 1*model_1_buy == 3000, "Ordered Model 1 Volume")
prob += (1*model_2_make + 1*model_2_buy == 2000, "Ordered Model 2 Volume")
prob += (1*model_3_make + 1*model_3_buy == 900, "Ordered Model 3 Volume")

# Solve the problem
status = prob.solve(PULP_CBC_CMD(msg=0))

# Print the solution
print(f"Status: {LpStatus[status]}")
print(f"Minimum cost: {'${:,.2f}'.format(value(prob.objective))}")  ## formats automatically with commas at thou
for v in prob.variables():
    print(f'Number of {v.name} = {v.varValue + 0}') # + 0 ensures no negative zeros

Status: Optimal
Minimum cost: $453,300.00
Number of Model_1_to_buy = 0.0
Number of Model_1_to_make = 3000.0
Number of Model_2_to_buy = 1450.0
Number of Model_2_to_make = 550.0
Number of Model_3_to_buy = 0.0
Number of Model_3_to_make = 900.0


#### Formulation


In [45]:
prob.writeLP('Slip Ring Cost Min Problem') # this writes to a separate file

f = open("Slip Ring Cost Min Problem", "r")
print(f.read())

\* Slip_Ring_Cost_Min_Problem *\
Minimize
OBJ: 61 Model_1_to_buy + 50 Model_1_to_make + 97 Model_2_to_buy
 + 83 Model_2_to_make + 145 Model_3_to_buy + 130 Model_3_to_make
Subject To
Harnessing_capacity_hours: Model_1_to_make + 2 Model_2_to_make
 + Model_3_to_make <= 5000
Ordered_Model_1_Volume: Model_1_to_buy + Model_1_to_make = 3000
Ordered_Model_2_Volume: Model_2_to_buy + Model_2_to_make = 2000
Ordered_Model_3_Volume: Model_3_to_buy + Model_3_to_make = 900
Wiring_capacity_hours: 2 Model_1_to_make + 1.5 Model_2_to_make
 + 3 Model_3_to_make <= 10000
Bounds
 Model_1_to_buy <= 3000
 Model_1_to_make <= 3000
 Model_2_to_buy <= 2000
 Model_2_to_make <= 2000
 Model_3_to_buy <= 900
 Model_3_to_make <= 900
End

